# 

## Import Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import InputLayer, Conv2D, BatchNormalization, MaxPooling2D, GlobalMaxPooling2D, Dense, Dropout
import matplotlib.pyplot as plt
import h5py

## Parameter Setup

In [ ]:
BATCH_SIZE = 32
EPOCHS = 200

model = str(input('socofing / family : '))


## Function for CNN Model

In [ ]:
# Function to create CNN model
def create_cnn_model(input_shape):
    model = Sequential()
    
    model.add(InputLayer(input_shape=input_shape))
    
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.15))
    
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.15))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.15))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.15))
    
    model.add(GlobalMaxPooling2D())
    
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.summary()
    return model

## Function for Model Training

In [ ]:
def train_model(X_train, y_train, X_val, y_val, input_shape, model_name, model):
    # Count male and female data in the training data
    male_count = np.sum(np.argmax(y_train, axis=1) == gender_dict['M'])
    female_count = np.sum(np.argmax(y_train, axis=1) == gender_dict['F'])
    print("Number of male data:", male_count)
    print("Number of female data:", female_count)

    # Model creation and compilation
    model = create_cnn_model(input_shape)
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                     factor=0.1,
                                                     patience=20,
                                                     min_lr=1e-7)

    # Training the model without data augmentation
    history = model.fit(X_train, y_train,
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=(X_val, y_val),
                        callbacks=[reduce_lr])

    # Save the model
    model.save(r'C:\Skripsi\FingeprintClassification\model_res\{}-fix.h5'.format(model_name))

    # Save the history to HDF5 file
    history_file_path = r'C:\Skripsi\FingeprintClassification\model_res\{}_history-fix.h5'.format(model_name)
    with h5py.File(history_file_path, 'w') as hf:
        # Create a group to store the history data
        history_group = hf.create_group('history')

        # Store the history data as datasets within the group
        for key, value in history.history.items():
            history_group.create_dataset(key, data=value)

    return history, model

## Function for Classification Report

In [ ]:
def print_classification_report(y_true, y_pred, target_names, title):
    report = classification_report(y_true, y_pred, target_names=target_names)
    print(title)
    print(report)